# The Trend of Local Temp Vs. Global Temp #

#### As the project required, I will
    1. Download local temperature data and global temp data from online serve by using SQL language.
    2. Open the data by using data analysing softwear (I'll use python here)
    3. Make a line chart, by using "moving average" method, to plot the trend of temp for local city and global temp.
    4. Then analyze the data and provide some clusions finally.

#### To download the  data by using SQL language, I used the code as below:
```
SELECT *
    FROM city_data
    WHERE city LIKE 'Shanghai';

SELECT *
    FROM global_data;
```        
- - -
The code below is the way I dealed with data

In [3]:
import numpy as np
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [4]:
df_sh = pd.read_csv(r'sh.csv') # this requires to put the code under the same path as these .csv data
df_gl = pd.read_csv(r'gl.csv')

In [5]:
df_sh.tail(5)

,year,city,country,avg_temp
168,2009,Shanghai,China,17.01
169,2010,Shanghai,China,16.75
170,2011,Shanghai,China,16.52
171,2012,Shanghai,China,16.47
172,2013,Shanghai,China,17.86


In [6]:
def avgList(df,timeWindow=10):
    avgTemp = []
    for i in df.index[timeWindow:]:
        avg_temp = df['avg_temp'][i-timeWindow:i].mean()
        avgTemp.append(avg_temp)
    return avgTemp

In [7]:
timeWindow = 10
sh_temp = np.array(avgList(df_sh,timeWindow))
gl_temp = np.array(avgList(df_gl,timeWindow))
sh_year = df_sh['year'][timeWindow:]
gl_year = df_gl['year'][timeWindow:]
diff = sh_temp[:] - gl_temp[(len(gl_temp) - len(sh_temp))-2:-2]
# Due to different "last year" in sh_year (2013), and in gl_year (2015), here make a little adjustment (use extra "-2")

In [17]:
trace_sh = go.Scatter(x=sh_year, y=sh_temp,
                      name = 'Shanghai Temp, C',
                      mode = 'lines')
trace_gl = go.Scatter(x=gl_year, y=gl_temp,
                      name = 'Global Temp, C',
                      mode = 'lines')
trace_diff = go.Scatter(x=sh_year, y=diff,
                      name = 'Difference, C',
                      mode = 'lines')
layout = go.Layout(title = 'Local Temp Compare with Global Temp',
                  titlefont=dict(size=18),
                  xaxis=dict(title='Year (timeWindow, 10 years)'),
                   margin=dict(l=150,t=50,r=150,b=50),
                   legend=dict(x=0.1,y=0.9),
                   yaxis=dict(title='Temp, C',range=[5,20]))

In [18]:
fig = go.Figure(data=[trace_sh,trace_gl,trace_diff],layout=layout)
py.iplot(fig, filename='Local Temp vs Global Temp')

In [10]:
print('The average of difference: %.2f' % np.mean(diff))
print('The 95%% confidence scope: (%.2f, %.2f)' % (np.mean(diff)-2*np.std(diff,ddof=1), np.mean(diff)+2*np.std(diff,ddof=1)))

The average of difference: 7.45
The 95% confidence scope: (7.22, 7.67)


 - - -
 #### Based on the analysis above, we can get conclusions as below:
     1. The local temperature (SH) as well as the global temperature (GL) rised about 1.5C since 1850.
     2. The temp of SH is ~7.45C higher than temp of GL.
     3. The temp rised faster since 1980s.
     4. Due to hypothesis testing, we consider the temp rising speed between SH and GL has no significant difference.